In [1]:
import os, time, h5py, platform, json, sys, matplotlib
if platform.system() == 'Linux':
    import matplotlib
    matplotlib.use('Agg')
sys.path.append("/Users/gkoolstra/Documents/Code")
sys.path.append(r"C:\Users\Gerwin\Documents\Code")
from matplotlib import pyplot as plt
from shutil import copyfile
import numpy as np
from tqdm import tqdm, trange
from scipy.optimize import minimize
from scipy.signal import savgol_filter
from termcolor import cprint
from glob import glob
from Common import common, kfit
import matplotlib.gridspec as gridspec
from TrapAnalysis import trap_analysis, artificial_anneal as anneal
%matplotlib inline
%config InlineBackend.figure_format='retina'

try:
    sys.path.append(r"S:\_Data\170422 - EonHe M018V6 with L3 etch\experiment")
    from data_cache import dataCacheProxy
except:
    pass

**Relevant references**
* Experiment: https://journals.aps.org/prb/pdf/10.1103/PhysRevB.79.045406
* Numerics: https://journals.aps.org/pre/abstract/10.1103/PhysRevE.65.046602

In [18]:
from scipy.constants import electron_mass as m_e, elementary_charge as qe

Vtg = np.linspace(0, -0.40, 0.00)
Vbar = 21E-3
Vtg_1 = -0.305
beta = -Vbar/Vtg_1

eps0 = 8.85E-12
epsHe = 1.056
omega0 = 2 * np.pi * 25E9

E0_e = (m_e * omega0**2 * qe / (2 * (eps0 * epsHe)**2 * (4 * np.pi)**2)) ** (1 / 3.)

Vb = Vbar + beta * Vtg

Vtg_2 = Vtg_1 + 0.75 * E0_e / beta
Vtg_3 = Vtg_1 + 1.31037 * E0_e / beta
Vtg_4 = Vtg_1 + 1.83545 * E0_e / beta

In [19]:
print(Vtg_1, Vtg_2, Vtg_3, Vtg_4)

-0.305 -0.24974136511343953 -0.20845432347159704 -0.16976738479661682


Fit the data to a model with $f_e$ as the only fit parameter. $V_\mathrm{bar}$ is chosen from the electrostatic simulation.

In [37]:
measured_jump_points = np.array([-0.305, -0.246, -0.2024, -0.1683]) # 1, 2, 3, 4

def fitfunc(x, *p):
    fe, = p
    
    omegae = 2 * np.pi * fe
    E0_e = (m_e * omegae**2 * qe / (2 * (eps0 * epsHe)**2 * (4 * np.pi)**2)) ** (1 / 3.)
    Vtg = np.linspace(0, -0.40, 0.00)
    Vbar = 22E-3
    Vtg_1 = -0.305
    beta = -Vbar/Vtg_1
    
    Vtg_2 = Vtg_1 + 0.75 * E0_e / beta
    Vtg_3 = Vtg_1 + 1.31037 * E0_e / beta
    Vtg_4 = Vtg_1 + 1.83545 * E0_e / beta
    
    return np.array([Vtg_1, Vtg_2, Vtg_3, Vtg_4])

The fit result is the frequency of the trap that best matches the four unloading voltages. The experimental data points are given in `measured_jump_points`

In [39]:
fr, ferr = kfit.fitbetter(None, measured_jump_points, fitfunc=fitfunc, fitparams=np.array([10E9]))
fr

array([  2.81150998e+10])

We can then obtain the unloading voltages, using the best fit value of $f_e$

In [40]:
fitfunc(0, *fr)

array([-0.305     , -0.24795777, -0.20533809, -0.16540245])